# Visualization of OVK data

First, read the data from various anonymized files and store them in six lists: 

1. mails: text of the emails
2. senders: sender of a mail: counselor or client
3. nbrOfChars: length of a mail in characters
4. counselor: id of the counselor
5. ids: id of the client
6. treatment type: either AS (auto-biographic writing) or ES (expressive writing)

For comparison, we include two additional data sets beside the two OVK data sets (AS and ES): biblical text from 1888 and newspaper text from 1985 (NRC). Each article is treated as an email (list mails). Dummy values have been used for the other five data lists.

In [ ]:
import csv
import os
import re
import sys
import xml.etree.ElementTree as ET

ASFILE = "../usb/ovk/data/eriktks/AS/text/AS-mails.csv"
ESFILE = "../usb/ovk/data/eriktks/ES/text/ES-mails.csv"
BIBLE = "../usb/ovk/data/eriktks/othertexts/bible.csv"
NEWSGAC = "../usb/ovk/data/eriktks/othertexts/newsgac.csv"
TACTUS = "../usb/output/emails-all.csv"
INFILENAMES = [ ASFILE,ESFILE ] # [ ASFILE,ESFILE,BIBLE,NEWSGAC ] [ TACTUS ]
CLIENT = "CLIENT"
BIBLE = "BIBLE"
NEWSPAPER = "NEWSPAPER"
COUNSELOR = "counselor"
ID = "client-id"
NBROFCHARS = "nbrOfCharsInWords"
NBROFSENTS = "nbrOfSents"
NBROFWORDS = "nbrOfWords"
SENDER = "sender"
SEPARATOR = ","
TEXT = "text"
MINWORDS = 0

(counselors,ids,mails,nbrOfChars,senders,treatments) = ([],[],[],[],[],[])
for inFileName in INFILENAMES:
    try: inFile = open(inFileName,"r")
    except Exception as e: sys.exit("cannot read file "+inFileName+": "+str(e))
    csvReader = csv.DictReader(inFile,delimiter=SEPARATOR)
    for row in csvReader:
        try:
            if True:
                if (inFileName == TACTUS or int(row[NBROFWORDS]) > MINWORDS) and row[SENDER] == "CLIENT":
                    mails.append(row[TEXT])
                    senders.append(row[SENDER])
                    if inFileName != TACTUS:
                        nbrOfChars.append(int(row[NBROFCHARS]))
                        counselors.append(row[COUNSELOR])
                        #ids.append(row[ID]+"-"+row[COUNSELOR]+"-"+row[SENDER]+"-"+str(len(ids)))
                        ids.append(row[ID])
                    else:
                        #ids.append(row[ID]+"-"+"-"+row[SENDER]+"-"+str(len(ids)))
                        ids.append(row[ID])
                    if inFileName == ASFILE: treatments.append("AS")
                    else: treatments.append("ES")
        except: sys.exit("unexpected row in file "+INFILENAME+": "+str(row))
    inFile.close()

Check if we want to have separate emails or have all emails for one person collapsed in one text

In [ ]:
COLLAPSE = True

if COLLAPSE:
    (newCounselors,newIds,newMails,newNbrOfChars,newSenders,newTreatments) = ([],[],[],[],[],[])
    collapse = {}
    for i in range(0,len(ids)):
        if ids[i] in collapse:
            thisId = collapse[ids[i]]
            newMails[thisId] += " "+mails[i]
            newNbrOfChars[thisId] += nbrOfChars[i]
        else:
            collapse[ids[i]] = len(newIds)
            if i < len(counselors): newCounselors.append(counselors[i])
            newIds.append(ids[i])
            newMails.append(mails[i])
            if i < len(nbrOfChars): newNbrOfChars.append(nbrOfChars[i])
            newSenders.append(senders[i])
            newTreatments.append(treatments[i])
    counselors = newCounselors
    ids = newIds
    mails = newMails
    nbrOfChars = newNbrOfChars
    senders = newSenders
    treatments = newTreatments

Read the meta data: exit phrase and the scores CESD and MHC. If the exit phrase is non-empty the patient has abandoned the treatment. The lower the CESD (depression) score, the better. The higher the MHC (mental health) score, the better.

In [ ]:
OVKMETAFILE = "../usb/ovk/data/eriktks/spss/opve.csv"
IDFIELDNAME = "onderzoeksnummer1"
EXITFIELDNAME = "Redenstoppen"

exitData = {}
cesdDiff = {}
mhcDiff = {}
try: inFile = open(OVKMETAFILE,"r")
except Exception as e: sys.exit("cannot read file "+OVKMETAFILE+": "+str(e))
csvReader = csv.DictReader(inFile,delimiter=SEPARATOR)
for row in csvReader: 
    exitData[row[IDFIELDNAME]] = row[EXITFIELDNAME]
    if row["CESD_TOT_t0"] != "NA" and row["CESD_TOT_t1"] != "NA": 
        cesdDiff[row[IDFIELDNAME]] = int(row["CESD_TOT_t1"])-int(row["CESD_TOT_t0"])
    if row["MHCtot_t0"] != "NA" and row["MHCtot_t1"] != "NA": 
        mhcDiff[row[IDFIELDNAME]] = float(row["MHCtot_t1"])-float(row["MHCtot_t0"])
inFile.close()

In [ ]:
#for i in cesdDiff: print(cesdDiff[i])

Next, count the tokens in the mails. We use a standard Python library for this, TfidfVectorizer, which normalizes the counts with respect to the lengths of the mails and prefers tokens that appear in a few mails over tokens that appear in every mail.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

NGRAMMIN = 1
NGRAMMAX = 2

tfidf_vectorizer = TfidfVectorizer(max_df=0.8,max_features=200000,min_df=0.2,use_idf=True,ngram_range=(NGRAMMIN,NGRAMMAX))
tfidf_matrix = tfidf_vectorizer.fit_transform(mails)

The AS and ES contain 2,000 mails and 25,000 different tokens. Therefore the previous analysis resulted in a table with 2,000 rows and 25,000 columns. We use principal component analysis to summarize this table to 2,000 rows and 4 columns.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4)
pca.fit(tfidf_matrix.toarray())
newSpace = pca.transform(tfidf_matrix.toarray())

Next the mails can be shown in a graph. Each row in the table corresponds with a mail. The values in the columns can be used as x-coordinates and y-coordinates to position the mails in the graph. We use the first two column values because they are expected to contain the most important information for creating interesting groups of mails in the graph.

In [ ]:
# draw graph of pca data: red: from client; blue: from counselor

%matplotlib notebook

import matplotlib.pyplot as plt
import random
import re

EXPERIMENT = "Emails"
THRESHOLD = 5
RANDOMFACTOR=0.00
(idsCli,idsCou,idsBib,idsNew,xCli,xCou,xBib,xNew,yCli,yCou,yBib,yNew) = ([],[],[],[],[],[],[],[],[],[],[],[])

def testFunction(thisId):
    return(stoppedTreatment(thisId))

def stoppedTreatment(thisId):
    return(re.search(r"[a-zA-Z]",exitData[thisId]))

def moreDepressed(thisId):
    return(ids[i] in cesdDiff and cesdDiff[ids[i]] > 0)

def worseMental(thisId):
    return(ids[i] in mhcDiff and mhcDiff[ids[i]] < 0)

random.seed()
for i in range(0,len(newSpace)):
    if senders[i] == "CLIENT" and not testFunction(ids[i]): # stoppedTreatment(ids[i]):
        xCli.append(newSpace[i][0]+random.random()*RANDOMFACTOR)
        yCli.append(newSpace[i][1]+random.random()*RANDOMFACTOR)
        idsCli.append(ids[i])
    # top left counselors: C16 C88 C77 C65
    elif senders[i] == "BIBLE":
        xBib.append(newSpace[i][0]+random.random()*RANDOMFACTOR)
        yBib.append(newSpace[i][1]+random.random()*RANDOMFACTOR)
        idsBib.append(ids[i])
    elif senders[i] == "NEWSPAPER":
        xNew.append(newSpace[i][0]+random.random()*RANDOMFACTOR)
        yNew.append(newSpace[i][1]+random.random()*RANDOMFACTOR)
        idsNew.append(ids[i])
    else: # senders[i] == CLIENT and exitData non-empty
        xCou.append(newSpace[i][0]+random.random()*RANDOMFACTOR)
        yCou.append(newSpace[i][1]+random.random()*RANDOMFACTOR)
        idsCou.append(ids[i])

def pickScatter(event):
    dataIds = []
    for i in range(0,len(event.ind)): 
        if event.artist == cliDots: dataIds.append(idsCli[event.ind[i]])
        elif event.artist == bibDots: dataIds.append(idsBib[event.ind[i]])
        elif event.artist == newDots: dataIds.append(idsNew[event.ind[i]])
        else: dataIds.append(idsCou[event.ind[i]])
    plt.gca().set_title(EXPERIMENT+str(dataIds),fontsize=12)

def makePlot(colorCli,colorCou,colorBib,colorNew):
    fig = plt.figure(figsize=(9,5))
    plt.gca().set_title(EXPERIMENT+": Click in the graph to see the relevant document ids")
    if len(xCou) > 0: couDots = plt.scatter(xCou,yCou,s=1,color=colorCou,picker=2, \
                      label="stopped treatment ("+str(len(idsCou))+")".format(THRESHOLD))
    else: couDots = plt.scatter(xCou,yCou,s=1,color=colorCou,picker=2)
    if len(xCli) > 0: cliDots = plt.scatter(xCli,yCli,s=1,color=colorCli,picker=2, \
                          label="completed treatment ("+str(len(idsCli))+")".format(THRESHOLD))
    else: cliDots = plt.scatter(xCli,yCli,s=1,color=colorCli,picker=2)
    if len(xBib) > 0:  bibDots = plt.scatter(xBib,yBib,s=1,color=colorBib,picker=2, \
                       label="from Bible ("+str(len(idsBib))+")".format(THRESHOLD))
    else: bibDots = plt.scatter(xBib,yBib,s=1,color=colorBib,picker=2)
    if len(xNew) > 0: newDots = plt.scatter(xNew,yNew,s=1,color=colorNew,picker=2, \
                      label="from newspaper ("+str(len(idsNew))+")".format(THRESHOLD))
    else: newDots = plt.scatter(xNew,yNew,s=1,color=colorNew,picker=2)
    plt.legend(fontsize=8)
    plt.gcf().canvas.mpl_connect("pick_event",pickScatter)
    plt.savefig("image.png")

makePlot("red","blue","green","black")


The cloud is divided in two part: clients that completed the treatment and clients that stopped the treatment. We examine the vocabulary of the two parts to find out what the differences are between the two parts. 

In [ ]:
# plot distribution of article lengths in words
# add ot, newspaper
# collect counselor texts of pca1 > 0.1, pca1 < 0.1; compute t-scores

THRESHOLD = 0.0

def getTokens(text):
    unigrams = text.split()
    ngrams = []
    for ngramSize in range(NGRAMMIN,NGRAMMAX+1):
        for i in range(0,len(unigrams)):
            if i+ngramSize-1 < len(unigrams):
                ngram = unigrams[i]
                for j in range(2,ngramSize+1):
                    ngram += " "+unigrams[i+j-1]
                ngrams.append(ngram)
    return(ngrams)

textCompleted = {}
textStopped = {}
totalCompleted = 0
totalStopped = 0
nbrOfMailsCompleted = 0
nbrOfMailsStopped = 0
for i in range(0,len(newSpace)):
    tokens = getTokens(mails[i])
    x = newSpace[i][0]
    y = newSpace[i][1]
    if x+y < 0.3: # not testFunction(ids[i]):
        nbrOfMailsCompleted += 1
        for i in range(0,len(tokens)):
            if i < len(tokens):
               ngram = tokens[i]
               if not ngram in textCompleted: textCompleted[ngram] = 0
               textCompleted[ngram] += 1
               totalCompleted += 1
    else:
        nbrOfMailsStopped += 1
        for i in range(0,len(tokens)):
            if i < len(tokens):
               ngram = tokens[i]
               if not ngram in textStopped: textStopped[ngram] = 0
               textStopped[ngram] += 1
               totalStopped += 1

Next we determine which words appear more often in the top-left group and which words appear more often in the bottom-right group. For this purpose we use the t-score, a statistical measure for comparing frequencies in two data sets.

In [ ]:
import math

tscores = {}
for token in textCompleted:
    freqCompleted = textCompleted[token]/totalCompleted
    seCompleted = freqCompleted/totalCompleted
    if token in textStopped:
        freqStopped = textStopped[token]/totalStopped
    else:
        freqStopped = 0.5/totalStopped
    seStopped = freqStopped/totalStopped
    tscores[token] = (freqCompleted-freqStopped)/math.sqrt(seCompleted+seStopped)

Finally, we display the top ten most specific words of each group

In [ ]:
import operator
import re

N = 20
sortedTscores = sorted(tscores.items(), key=operator.itemgetter(1), reverse=True)

print("COMPLETED TREATMENT ("+str(nbrOfMailsCompleted)+")")
shown = 0
for i in range(0,len(sortedTscores)):
    if re.match(r"^[a-zA-Z ]+$",sortedTscores[i][0]):
        print(str(shown+1)+". "+sortedTscores[i][0])
        shown += 1
    if shown >= N: break

print("\nSTOPPED TREATMENT ("+str(nbrOfMailsStopped)+")")
shown = 0
for i in range(len(sortedTscores)-1,-1,-1):
    if re.match(r"^[a-zA-Z ]+$",sortedTscores[i][0]):
        print(str(shown+1)+". "+sortedTscores[i][0])
        shown += 1
    if shown >= N: break

Next:

* explore linking mails of the same session (+)
* explore topic classification with nmf (-)
* solve click in graph bug
* check mails of stopped patients